# Signal Classifier

## Library Imports

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf
#np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
def plot(fig,x,y,name):
    fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name=name))


## Defintion of Different Signal Functions

In [3]:
def dft(N,M=40):
    #w=np.e**((2*np.pi*-1j)/N)
    power=np.zeros((M,N),dtype=complex)
    xj = np.linspace(-np.pi, np.pi, N)
    for i in range(M):
        for j in range(N):
            #power[i][j]=((-N/2)+i)*j
            power[i][j] = -1j*(i-M/2)*xj[j]
    #DFT=(w**po
    # wer)/N
    DFT = np.e**power
    return DFT
def box(x,a=2,b=5,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k))
        fkhat[k==0]=(2*a*b)
        fkhat[k!=0]=((2*b*np.sin(a*k[k!=0]))/k[k!=0])
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(box(x,a,b,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=b
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(box(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def saw(x,a=2,b=5,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=0
        fkhat[k!=0]=(2*1j*b*(np.sin(a*k[k!=0])-a*k[k!=0]*np.cos(a*k[k!=0])))/(k[k!=0]**2)
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(saw(x,a,b,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=-b*x[np.abs(x)<a]
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(saw(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def exp(x,a=2,b=2,c=-1,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=(2*((a*b*c)+np.sinh(a*b)))/b
        fkhat[k!=0]=((2*c*np.sin(a*k[k!=0]))/k[k!=0])+((2*np.sinh(a*(b+(1j*k[k!=0]))))/(b+(1j*k[k!=0])))
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(exp(x,a,b,c,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c+np.e**(-b*x[[np.abs(x)<a]])
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
           return y/np.max(abs(exp(x,a,b,c,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def sinu(x,a=2,b=2,c=-1,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==b]=((1j*c*np.sin(2*a*b))/(2*b))-(1j*a*c)
        fkhat[k==-b]=(1/2)*1j*c*((2*a)-((np.sin(2*a*b))/b))
        fkhat[k!=b]=(2*1j*c*((b*np.cos(a*b)*np.sin(a*k[k!=b]))-(k[k!=b]*np.sin(a*b)*np.cos(a*k[k!=b]))))/((b**2)-(k[k!=b]**2))
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(sinu(x,a,b,c,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c*np.sin(b*x[np.abs(x)<a])
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(sinu(x,a,b,c,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def gaus(x,a=2,b=2,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40,M=40):
    if fourier:
        fkhat=((np.dot(dft(N,M),gaus(x,a,b)))/N)
        if normalized:
            return fkhat/np.max(abs(gaus(x,a,b,normalized=False)))
        return fkhat
    y=np.zeros(len(x))
    y[np.abs(x)<a]=np.e**(-a*(x[np.abs(x)<a]**(2*b)))
    y=np.nan_to_num(y)
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(gaus(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def FourierSeries(cn,X,N):
    fx=[]
    '''
    Cn=np.zeros(len(cn))
    Cn[0]=cn[int(N/2)]
    Cn[1:int(N/2)-1]=cn[1:int(N/2)-1]
    Cn[int(N/2)+1:N]=cn[int(N/2)+1:N]
    fx=np.fft.ifft(Cn)
    '''
    
    for x in X:
        result=0
        for i in range(int((-N/2)),int((N/2))):
            result=result+cn[int(i+(N/2))]*(np.e**(1j*i*x)) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    return fx

In [4]:
SignalsDict={'Box':1,'Saw':2,'Exp':3,'Sin':4,'Gaus':5}
def Generate(signal,x,Amount,normalized=True,fourier=False,N=40,M=40,Noise=False,noiseParameter=0.1):
    signalOutput=[]
    length=Amount+len(x)
    print(SignalsDict[signal])
    if(SignalsDict[signal]==1):
        #a=np.linspace(np.pi/4,np.pi/2,len(x))
        a=np.linspace(.10,2.90,length)
        b=np.append(np.linspace(-100,-0.01,int(length/2)),np.linspace(0.01,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=box(x,a[i],b[i],normalized=normalized,fourier=True,N=N)
                normalization=1#np.max(abs(F))
                #BoxSignal.append(F/normalization)
                #BoxSignalFourier.append(fx)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=box(x,a[i],b[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=box(x,a[i],b[i],normalized=normalized)
                #normalization=1#np.max(abs(F))
                #BoxSignal.append(F/normalization)
                #BoxSignalFourier.append(fx)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
    elif(SignalsDict[signal]==2):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-100,-0.01,int(length/2)),np.linspace(0.01,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=saw(x,a[i],b[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=saw(x,a[i],b[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=saw(x,a[i],b[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
    elif(SignalsDict[signal]==3):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-1,-0.1,int(length/2)),np.linspace(0.1,1,int(length/2)))
        c=np.append(np.linspace(-3,-1.01,int(length/2)),np.linspace(-1.01,1,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        c=np.random.permutation(c)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=exp(x,a[i],b[i],c[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=exp(x,a[i],b[i],c[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        else:
            for i in range(Amount):
                F=exp(x,a[i],b[i],c[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
    elif(SignalsDict[signal]==4):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-2*np.pi,-0.3,int(length/2)),np.linspace(0.3,2*np.pi,int(length/2)))
        c=np.append(np.linspace(-100,-0.1,int(length/2)),np.linspace(0.1,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        c=np.random.permutation(c)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=sinu(x,a[i],b[i],c[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=sinu(x,a[i],b[i],c[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        else:
            for i in range(Amount):
                F=sinu(x,a[i],b[i],c[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
    elif(SignalsDict[signal]==5):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-10,-1,int(length/2)),np.linspace(1,10,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)

        if fourier:
            print(a[0])
            for i in range(Amount):
                x=np.linspace(-np.pi,np.pi,N)
                cn=gaus(x,a[i],int(b[i]),normalized=normalized,fourier=True,N=N,M=M)
                signalOutput.append(FourierSeries(cn,x,M))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=gaus(x,a[i],int(b[i]),noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=gaus(x,a[i],int(b[i]),normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]

In [67]:
fig=go.Figure()
fig.update_xaxes(range=[-np.pi,np.pi])
x=np.linspace(-np.pi,np.pi,100)
Generate('Box',x,Amount=2)
plot(fig,x,Generate('Box',x,Amount=1,normalized=True,fourier=True,N=100)[0],'Box')
plot(fig,x,Generate('Saw',x,Amount=1,normalized=True,Noise=True,noiseParameter=1.4)[0],'Saw')
plot(fig,x,Generate('Exp',x,Amount=1,normalized=True)[0],'Exp')
plot(fig,x,Generate('Sin',x,Amount=1,normalized=True,Noise=True,noiseParameter=0.5)[0],'Sin')
plot(fig,x,Generate('Gaus',x,Amount=100,normalized=True,Noise=True,N=80,M=40)[0],'Gaus')

fig

1
1
1.556
2
3
4
5


## Testing
###

In [128]:

BoxSignal=Generate('Box',x,Amount=100)
SawSignal=Generate('Saw',x,Amount=100)
ExpSignal=Generate('Exp',x,Amount=100)
SinSignal=Generate('Sin',x,Amount=100)
GausSignal=Generate('Gaus',x,Amount=100)

#parameters




1
2
3
4
5


In [70]:
np.shape(BoxSignal)

(100, 100)

In [129]:
Signals=[BoxSignal[0],SawSignal[0],ExpSignal[0],SinSignal[0],GausSignal[0]]
SignalLabels=[
    1*np.ones(100),
    2*np.ones(100),
    3*np.ones(100),
    4*np.ones(100),
    5*np.ones(100)]
SignalsTrain=np.array(np.concatenate(Signals).flat)
SignalsTrain=SignalsTrain.reshape((500,100))
SignalLabelsTrain=np.array(np.concatenate(SignalLabels).flat)

In [130]:
np.shape(SignalsTrain)

(500, 100)

In [131]:
modelNN = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6)
])
modelNN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelNN.fit(SignalsTrain, SignalLabelsTrain, epochs=100)


Epoch 1/100
16/16 [==============================] - 1s 4ms/step - loss: 1.5937 - accuracy: 0.4100
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: 1.2157 - accuracy: 0.6300
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 1.0209 - accuracy: 0.6760
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.8929 - accuracy: 0.7100
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.8047 - accuracy: 0.7360
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.7370 - accuracy: 0.7780
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6848 - accuracy: 0.7880
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6394 - accuracy: 0.8120
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6046 - accuracy: 0.8080
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5713 - accuracy: 0.8280
Epoch 11/

In [140]:

BoxSignal=Generate('Box',x,Amount=1000)
SawSignal=Generate('Saw',x,Amount=1000)
ExpSignal=Generate('Exp',x,Amount=1000)
SinSignal=Generate('Sin',x,Amount=1000)
GausSignal=Generate('Gaus',x,Amount=1000)
a=[BoxSignal[1],SawSignal[1],ExpSignal[1],SinSignal[1],GausSignal[1]]
b=[BoxSignal[2],SawSignal[2],ExpSignal[2],SinSignal[2],GausSignal[2]]
c=[np.zeros(1000),np.zeros(1000),ExpSignal[3],SinSignal[3],np.zeros(1000)]
Signals=[BoxSignal[0],SawSignal[0],ExpSignal[0],SinSignal[0],GausSignal[0]]
SignalLabels=[
    1*np.ones(1000),
    2*np.ones(1000),
    3*np.ones(1000),
    4*np.ones(1000),
    5*np.ones(1000)]
SignalsTest=np.array(np.concatenate(Signals).flat)
a=np.array(np.concatenate(a).flat)
b=np.array(np.concatenate(b).flat)
c=np.array(np.concatenate(c).flat)

SignalsTest=SignalsTest.reshape((5000,100))
SignalLabelsTest=np.array(np.concatenate(SignalLabels).flat)

1
2
3
4
5


In [142]:
test_loss, woNoiseAcc = modelNN.evaluate(SignalsTest,  SignalLabelsTest, verbose=2,use_multiprocessing=True,)
#modelNN.evaluate()
#modelNN.predict_batch(BoxSignal[0])
probability_model=tf.keras.Sequential([modelNN,tf.keras.layers.Softmax()])
predictions=probability_model.predict(SignalsTest)

157/157 - 0s - loss: 0.1895 - accuracy: 0.9390


In [145]:
len(predictions)

5000

In [147]:
testResults=[]
for i in range(len(predictions)):
    a=np.max(predictions[i])
    maxElement=np.where(predictions[i]==a)
    testResults.append(maxElement[0][0])


In [149]:
len(testResults)

5000

In [150]:
Model1Summ=pd.DataFrame()
Model1Summ['Actual Classifcation']=SignalLabelsTest
Model1Summ['Predicted Classification']=testResults
Model1Summ['a']=a
Model1Summ['b']=b
Model1Summ['c']=c


In [153]:
Model1Summ.to_csv('Model1Results.csv')

In [155]:
modelNN.save('Model1TrainedTF')

INFO:tensorflow:Assets written to: Model1TrainedTF/assets


### Box Function

[array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 

### Saw Function

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawSignal=[]
SawSignalFourier=[]
#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(10000):
    F=saw(x,a[i],b[i])
    normalization=np.max(abs(F))
    SawSignal.append(F/normalization)
    cn=SawFourierAnalytical(a[i],b[i],N)
    fx=FourierSeries(cn,x,N)
    SawSignalFourier.append(fx)

### Exponential

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.1,5000),np.linspace(0.1,1,5000))
c=np.append(np.linspace(-3,-1.01,5000),np.linspace(-1.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpSignal=[]
ExpSignalFourier=[]

for i in range(10000):
    F=exp(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    ExpSignal.append(F/normalization)
    cn=ExpFourierAnalytical(a[i],b[i],c[i],N)
    fx=FourierSeries(cn,x,N)
    ExpSignalFourier.append(fx)

### Sinusoid

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.1,5000),np.linspace(0.1,2*np.pi,5000))
c=np.append(np.linspace(-1,-0.1,5000),np.linspace(0.1,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinSignal=[]
SinSignalFourier=[]
for i in range(10000):
    F=sinu(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    SinSignal.append(sinu(x,a[i],b[i],c[i])/normalization)
    cn=SinuFourierAnalytical(a[i],b[i],b[i],N)
    fx=FourierSeries(cn,x,N)
    SinSignalFourier.append(fx)

### Gaussian

In [13]:
#a=np.linspace(0.01,3,10000)
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.linspace(1,10,10000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausSignal=[]
for i in range(10000):
    F=gaus(x,a[i],int(b[i]))
    normalization=np.max(abs(F))
    cn=(np.dot(dft(N,100),gaus(x,a,b))/N)
    fx=FourierSeries(cn,x,100)
    GausSignal.append(F/normalization)

NameError: name 'N' is not defined

### Combining Results

In [ ]:
Signals=[BoxSignal,SawSignal,ExpSignal,SinSignal,GausSignal]
SignalLabels=[
    np.zeros(20000),
    1*np.ones(20000),
    2*np.ones(20000),
    3*np.ones(20000),
    4*np.ones(20000)]
SignalsTrain=np.array(np.concatenate(Signals).flat)
SignalsTrain=SignalsTrain.reshape((100000,100))
SignalLabelsTrain=np.array(np.concatenate(SignalLabels).flat)

## Generating 1000 Functions + Fourier Series Representation of each signal for Testing Model

### Box Function

In [ ]:
## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxSignal=[]
for i in range(1000):
    F=box(x,a[i],b[i])
    normalization=np.max(abs(F))
    BoxSignal.append(F/normalization)






#### Fourier Representation


In [ ]:

def e(n,x,positve=True):
    if(positve):
        return np.e**(1j*n*x)
    else:
        return np.e**(-1j*n*x)
''' TOO SLOW for COLAB
def calculateCn(M1,D,F):
    for p in range(M1-1):
        for q in range(M1-1):
            D[p][q]=(e(n[p],xi[q],False))
    cn=np.dot(D,F)/M1
    return cn
  ##Fourier Series Representation

  M1=200 #number of grid points
  M2 = 1500
  h=np.linspace(0,M1-1,M1-1) #
  xi=-np.pi+(2*np.pi/M1)*h
  D=np.zeros((M1-1,M1-1),dtype=complex)
  n=np.linspace(-M1/2,M1/2,M1-1)
  F=box(xi,a[i],b[i])
  cn=calculateCn(M1,D,F)
  h2=np.linspace(0,M2-1,M2-1)
  xi2 =-np.pi+ (2*np.pi/M2)*h2
  xi=np.linspace(-np.pi,np.pi,100)
  D2=np.zeros((M2-1,M1-1),dtype=complex)W
  for p in range(M2-1):
    for q in range(M1-1):
      D2[p][q]=(e(xi2[p],n[q]))

  fx2 = (np.dot(D2,cn)).real
  FourierBox.append(fx2)

#TODO
#figure out fft


fx=[]
for xi in x:
    result=0
    for n in range(int((-N/2)),int((N/2))):
      result=result+(cn[int(i+(N/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
    fx.append(result.real)
fig=go.Figure()
plot(fig,x,box(x),'')
plot(fig,x,fx,'')
fig
'''

In [ ]:

def fft(N,M=40):
    #w=np.e**((2*np.pi*-1j)/N)
    power=np.zeros((M,N),dtype=complex)
    xj = np.linspace(-np.pi, np.pi, N)
    for i in range(M):
        for j in range(N):
            #power[i][j]=((-N/2)+i)*j
            power[i][j] = -1j*(i-M/2)*xj[j]
    #DFT=(w**power)/N
    DFT = np.e**power
    return DFT


In [ ]:

N=100
def f(x):
    return box(x)
x=np.linspace(-np.pi,np.pi,N)
fx=[]
gx=[]
cn=((np.dot(fft(N),f(x)))/N)
M =40

k=np.linspace(int((-M/2)),(M-1)-int((M/2)),M)
dn=cn*1j*np.sign(k)

for xi in x:
    result=0
    for n in range(int((-M/2)),(M-1)-int((M/2))):
      result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
    fx.append(result.real)
fig=go.Figure()
plot(fig,x,fx,'')
plot(fig,x,f(x),'original')
fig

#dn=cn*

In [ ]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxFourier40=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N),box(x,a[i],b[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    BoxFourier40.append(fx)



In [ ]:

N=100
M = 80
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxFourier80=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N,M),box(x,a[i],b[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    BoxFourier80.append(fx)



#### with noise

In [ ]:
## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxNoiseLow=[]
BoxNoiseHigh=[]
for i in range(1000):
    F=box(x,a[i],b[i],True,0.05)
    normalization=np.max(abs(F))
    BoxNoiseLow.append(F/normalization)
    F=box(x,a[i],b[i],True,0.3)
    normalization=np.max(abs(F))
    BoxNoiseHigh.append(F/normalization)



### Saw Function

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawSignal=[]
#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(1000):
    F=saw(x,a[i],b[i])
    normalization=np.max(abs(F))
    SawSignal.append(F/normalization)

#### Fourier

In [ ]:


N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawFourier40=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N),saw(x,a[i],b[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    SawFourier40.append(fx)


    


In [ ]:


N=100
M = 80
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawFourier80=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N,M),saw(x,a[i],b[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    SawFourier80.append(fx)


    


#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawNoiseHigh=[]
SawNoiseLow=[]

#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(1000):
    F=saw(x,a[i],b[i],True,.3)
    normalization=np.max(abs(F))
    SawNoiseHigh.append(F/normalization)
    F=saw(x,a[i],b[i],True,.05)
    normalization=np.max(abs(F))
    SawNoiseLow.append(F/normalization)

### Exponential

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpSignal=[]
for i in range(1000):
    F=exp(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    ExpSignal.append(F/normalization)

#### Fourier

In [ ]:
N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpFourier40=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N),exp(x,a[i],b[i],c[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    ExpFourier40.append(fx)


    


In [ ]:
N=100
M = 80
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpFourier80=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N,M),exp(x,a[i],b[i],c[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    ExpFourier80.append(fx)


    


#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpNoiseHigh=[]
ExpNoiseLow=[]

for i in range(1000):
    F=exp(x,a[i],b[i],True,0.05)
    normalization=np.max(abs(F))
    ExpNoiseLow.append(F/normalization)
    F=exp(x,a[i],b[i],True,.3)
    normalization=np.max(abs(F))
    ExpNoiseHigh.append(F/normalization)

### Sinusoid

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,2*np.pi,500))
c=np.append(np.linspace(-1,-0.1,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinSignal=[]
for i in range(1000):
    F=sinu(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    SinSignal.append(F/normalization)

#### Fourier

In [ ]:
N=100
M = 40

x=np.linspace(-np.pi,np.pi,N)
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,2*np.pi,500))
c=np.append(np.linspace(-1,-0.1,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinFourier40=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N),sinu(x,a[i],b[i],c[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    SinFourier40.append(fx)




In [ ]:
N=100
M = 80

x=np.linspace(-np.pi,np.pi,N)
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,2*np.pi,500))
c=np.append(np.linspace(-1,-0.1,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinFourier80=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N,M),sinu(x,a[i],b[i],c[i])))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    SinFourier80.append(fx)

#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.1,500),np.linspace(0.1,2*np.pi,500))
c=np.append(np.linspace(-1,-0.1,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinNoiseHigh=[]
SinNoiseLow=[]
for i in range(1000):
    F=sinu(x,a[i],b[i],c[i],True,.3)
    normalization=np.max(abs(F))
    SinNoiseHigh.append(F/normalization)
    F=sinu(x,a[i],b[i],c[i],True,.05)
    normalization=np.max(abs(F))
    SinNoiseLow.append(F/normalization)

### Gaussian

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(1,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausSignal=[]
for i in range(1000):
    F=gaus(x,a[i],int(b[i]))
    normalization=np.max(abs(F))
    GausSignal.append(F/normalization)

#### Fourier

In [ ]:
N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(1,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)
GausFourier40=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N),gaus(x,a[i],int(b[i]))))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    GausFourier40.append(fx)




In [ ]:
N=100
M = 80
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(1,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)
GausFourier80=[]
for i in range(1000):
    x=np.linspace(-np.pi,np.pi,N)
    fx=[]
    cn=((np.dot(fft(N,M),gaus(x,a[i],int(b[i]))))/N)
    for xi in x:
        result=0
        for n in range(int((-M/2)),(M-1)-int((M/2))):
            result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    GausFourier80.append(fx)




#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(1,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausNoiseHigh=[]
GausNoiseLow=[]
for i in range(1000):
    F=gaus(x,a[i],int(b[i]),True,0.3)
    normalization=np.max(abs(F))
    GausNoiseHigh.append(F/normalization)
    F=gaus(x,a[i],int(b[i]),True,0.05)
    normalization=np.max(abs(F))
    GausNoiseLow.append(F/normalization)

### Combining Results

In [ ]:
#Combining Signals
Signals=[BoxSignal,SawSignal,ExpSignal,SinSignal,GausSignal]
SignalsWNoiseHigh=[[BoxNoiseHigh,SawNoiseHigh,ExpNoiseHigh,SinNoiseHigh,GausNoiseHigh]]
SignalsWNoiseLow=[[BoxNoiseLow,SawNoiseLow,ExpNoiseLow,SinNoiseLow,GausNoiseLow]]
SignalsFourier40=[[BoxFourier40,SawFourier40,ExpFourier40,SinFourier40,GausFourier40]]
SignalsFourier80=[[BoxFourier80,SawFourier80,ExpFourier80,SinFourier80,GausFourier80]]
SignalLabels=[
    np.zeros(1000),
    1*np.ones(1000),
    2*np.ones(1000),
    3*np.ones(1000),
    4*np.ones(1000)]
SignalsTest=np.array(np.concatenate(Signals).flat)
SignalsTest=SignalsTest.reshape((5000,100))
SignalsWNoiseHighTest=np.array(np.concatenate(SignalsWNoiseHigh).flat)
SignalsWNoiseHighTest=SignalsWNoiseHighTest.reshape((5000,100))
SignalsWNoiseLowTest=np.array(np.concatenate(SignalsWNoiseLow).flat)
SignalsWNoiseLowTest=SignalsWNoiseLowTest.reshape((5000,100))

SignalsFourier40Test=np.array(np.concatenate(SignalsFourier40).flat)
SignalsFourier80Test=np.array(np.concatenate(SignalsFourier80).flat)

#SignalsFourierTest=SignalsFourierTest.reshape((5000,100))
SignalLabelsTest=np.array(np.concatenate(SignalLabels).flat)

## Visualization of Data of N samples from test Data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=5
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxSignal[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawSignal[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpSignal[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinSignal[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausSignal[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Generated Signals w/o noise"%N)
fig.show()



### Fourier Data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=10
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxFourier40[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawFourier40[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpFourier40[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinFourier40[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausFourier40[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Fourier Data N=40"%N)
fig.show()



In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=10
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxFourier80[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawFourier80[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpFourier80[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinFourier80[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausFourier80[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Fourier Data N=80"%N)
fig.show()



### With Noise

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=5
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxNoiseLow[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawNoiseLow[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpNoiseLow[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinNoiseLow[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausNoiseLow[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Generated Signals with Noise, LOW"%N)
fig.show()



In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=5
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxNoiseHigh[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawNoiseHigh[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpNoiseHigh[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinNoiseHigh[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausNoiseHigh[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Generated Signals with Noise, HIGH"%N)
fig.show()




## Training Model 

### Neural Network(100 epochs, 128 RELU activation layer)

In [ ]:
modelNN = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])
modelNN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelNN.fit(SignalsTrain, SignalLabelsTrain, epochs=100)


### Decision Tree Classifier

## Testing Model


In [ ]:
SignalsFourier40Test=np.resize(SignalsFourier40Test,(5000,100))

In [ ]:
SignalsFourier80Test=np.resize(SignalsFourier80Test,(5000,100))

### NN Model

In [ ]:
test_loss, woNoiseAcc = modelNN.evaluate(SignalsTest,  SignalLabelsTest, verbose=2)
test_loss, wnoiseHAcc = modelNN.evaluate(SignalsWNoiseHighTest,  SignalLabelsTest, verbose=2)
test_loss, wnoiseLAcc = modelNN.evaluate(SignalsWNoiseLowTest,  SignalLabelsTest, verbose=2)
test_loss, fourier40Acc = modelNN.evaluate(SignalsFourier40Test,  SignalLabelsTest, verbose=2)
test_loss, fourier80Acc = modelNN.evaluate(SignalsFourier80Test,  SignalLabelsTest, verbose=2)



print(
"Raw Signals: %.5f \nSignals with High Noise: %.5f\nSignals with Low Noise: %.5f\nFourier Represnetation of Signals N=40: %.5f\nFourier Represnetation of Signals N=80 %.5f"%(woNoiseAcc,wnoiseHAcc,wnoiseLAcc,fourier40Acc,fourier80Acc))
